In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 导入数据

In [71]:
data_train = pd.read_csv('E:\\tianchi\\titanic\\train.csv')
dada_test = pd.read_csv('E:\\tianchi\\titanic\\test.csv')

In [72]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [73]:
data_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 填充数据

In [74]:
#年龄用均值填充
data_train['Age'] = data_train['Age'].fillna(data_train['Age'].mean())

### 采用Pclass,Sex,Age,SibSp,Parch,Fare这六个特征建模（线性回归）

In [75]:
#特征处理，Pclass 数字化，Sex 数字化
data_train.loc[data_train['Sex'] == 'male','Sex'] = 1
data_train.loc[data_train['Sex'] == 'female','Sex'] = 2

In [76]:
#选取特征
features = ['Pclass','Sex','Age','SibSp','Parch','Fare']

In [77]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import model_selection

In [78]:
#初始化线性回归模型
lr = LinearRegression()
kf = KFold(n_splits = 3,shuffle=False,random_state=1)

#构造一个空list用于存储预测值
pred = []

#3折交叉验证
for train,test in kf.split(data_train):
    train_fea = data_train[features].iloc[train,:]
    train_tar = data_train['Survived'].iloc[train]
    lr.fit(train_fea,train_tar)
    pre = lr.predict(data_train[features].iloc[test,:])
    pred.append(pre)

#将预测值合并起来，并修改为0--1
pred = np.concatenate(pred,axis = 0)
pred[pred > 0.5] = 1
pred[pred <= 0.5] = 0

In [79]:
#计算准确率
accu = len(pred[pred == data_train['Survived']])/len(data_train['Survived'])
accu

0.7845117845117845

### Logistic Regression

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection

In [81]:
lr = LogisticRegression(random_state=1)
lr.fit(data_train[features],data_train['Survived'])
scores = model_selection.cross_val_score(lr,data_train[features],data_train['Survived'],cv = 3)
scores.mean()

0.7912457912457912

### 随机森林算法

In [82]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

In [84]:
#选取特征
features = ['Pclass','Sex','Age','SibSp','Parch','Fare']

#10课决策树，停止的条件：样本个数为2，叶子个数为1
rf = RandomForestClassifier(random_state=1,min_samples_split=2,min_samples_leaf=1)

kf = model_selection.KFold(n_splits = 3,shuffle = False,random_state = 1)

scores = model_selection.cross_val_score(rf,data_train[features],data_train['Survived'],cv = kf)

print (scores)
print (scores.mean())

[0.77104377 0.81144781 0.7979798 ]
0.7934904601571269


### 增加树的个数，采用10折交叉验证

In [85]:
#30颗数，停止的条件，样本个数为2，叶子个数为1
rf = RandomForestClassifier(random_state=1,min_samples_split=2,min_samples_leaf=1)

kf = model_selection.KFold(n_splits=10,shuffle=False,random_state=1)

scores = model_selection.cross_val_score(rf,data_train[features],data_train['Survived'],cv = kf)

print (scores)
print (scores.mean())

[0.75555556 0.82022472 0.76404494 0.84269663 0.87640449 0.86516854
 0.79775281 0.82022472 0.84269663 0.85393258]
0.8238701622971286
